In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 맷플롯립 한글 사용 가능
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

In [25]:
sp_2024_df = pd.read_csv('../4.Public_transportation/Subway_Number_of_passengers2024.csv',
            encoding = 'CP949')
ss_df = pd.read_csv('Data_Preprocessing/4_seoul_stations.csv')
sp_2024_df.columns = ['month', 'line', 'station_name', 
                      'paid_boarding', 'free_boarding', 
                      'paid_alighting', 'free_alighting', 'data_date']
sp_2024_df.info(), ss_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6826 entries, 0 to 6825
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   month           6826 non-null   int64 
 1   line            6826 non-null   object
 2   station_name    6826 non-null   object
 3   paid_boarding   6826 non-null   int64 
 4   free_boarding   6826 non-null   int64 
 5   paid_alighting  6826 non-null   int64 
 6   free_alighting  6826 non-null   int64 
 7   data_date       6826 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 426.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   line          274 non-null    object
 1   station_name  274 non-null    object
 2   district      274 non-null    object
dtypes: object(3)
memory usage: 6.5+ KB


(None, None)

In [26]:
merged_df = pd.merge(sp_2024_df, ss_df, how="inner", on="station_name")
merged_df = merged_df[['month', 'line_x', 'station_name', 'paid_boarding', 'free_boarding',
                       'paid_alighting', 'free_alighting', 'district']]
merged_df.columns = ['month', 'line', 'station_name', 'paid_boarding', 'free_boarding',
                     'paid_alighting', 'free_alighting', 'district']
merged_df.info()
merged_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4220 entries, 0 to 4219
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   month           4220 non-null   int64 
 1   line            4220 non-null   object
 2   station_name    4220 non-null   object
 3   paid_boarding   4220 non-null   int64 
 4   free_boarding   4220 non-null   int64 
 5   paid_alighting  4220 non-null   int64 
 6   free_alighting  4220 non-null   int64 
 7   district        4220 non-null   object
dtypes: int64(5), object(3)
memory usage: 263.9+ KB


,month,line,station_name,paid_boarding,free_boarding,paid_alighting,free_alighting,district
0,202410,1호선,동묘앞,169167,175134,177809,177898,종로구
1,202410,1호선,동묘앞,169167,175134,177809,177898,종로구
2,202410,1호선,시청,705315,140635,722224,139726,중구
3,202410,1호선,시청,705315,140635,722224,139726,중구
4,202410,1호선,종각,968038,168194,947571,153881,종로구


In [27]:
merged_df['total_boarding'] = merged_df['paid_boarding'] + merged_df['free_boarding']
merged_df['total_alighting'] = merged_df['paid_alighting'] + merged_df['free_alighting']

merged_df['paid_boarding_ratio'] = merged_df['paid_boarding'] / merged_df['total_boarding']
merged_df['free_boarding_ratio'] = merged_df['free_boarding'] / merged_df['total_boarding']
merged_df['paid_alighting_ratio'] = merged_df['paid_alighting'] / merged_df['total_alighting']
merged_df['free_alighting_ratio'] = merged_df['free_alighting'] / merged_df['total_alighting']

merged_df = merged_df[['month', 'district',
                       'paid_boarding', 'paid_boarding_ratio',
                       'free_boarding', 'free_boarding_ratio', 'total_boarding', 
                       'paid_alighting', 'paid_alighting_ratio',
                       'free_alighting', 'free_alighting_ratio', 'total_alighting']]

merged_df['month'] = pd.to_datetime(merged_df['month'], format='%Y%m')
merged_df['year'] = merged_df['month'].dt.year

In [28]:
grouped_by_month_df = merged_df.groupby(['month', 'district']).mean().reset_index()
grouped_by_year_df = merged_df.groupby('district').mean().reset_index()

# 지수 표기법 대신 일반 숫자로 표시 (소숫점 5자리까지)
pd.options.display.float_format = '{:.5f}'.format

grouped_by_month_df.head(), grouped_by_year_df.head()
# grouped_by_month_df.info(), grouped_by_year_df.info()

(       month district  paid_boarding  paid_boarding_ratio  free_boarding  \
 0 2024-01-01      강남구   543469.30769              0.85446    75063.76923   
 1 2024-01-01      강동구   274032.62500              0.75505    85493.62500   
 2 2024-01-01      강북구   547824.00000              0.76279   157437.00000   
 3 2024-01-01      강서구   311730.66667              0.82282    67628.50000   
 4 2024-01-01      관악구   763650.60000              0.81580   130890.00000   
 
    free_boarding_ratio  total_boarding  paid_alighting  paid_alighting_ratio  \
 0              0.14554    618533.07692    561260.34615               0.86036   
 1              0.24495    359526.25000    273155.87500               0.75367   
 2              0.23721    705261.00000    543908.50000               0.75648   
 3              0.17718    379359.16667    298828.41667               0.81710   
 4              0.18420    894540.60000    741579.20000               0.81967   
 
    free_alighting  free_alighting_ratio  total_